### CITIBIKE ANALYSIS PROJECT


#### Libraries

In [1]:
# Libraries for reading and manipulating data
import numpy as np
import pandas as pd

pd.set_option('display.max_columns',20)
pd.set_option('display.max_rows', 200)

### Reading Dataset


In [2]:
#read the csv file into a dataframe
data = pd.read_csv('/Users/drushti/Documents/PythonProject/CitiBike/citibike.csv')

#create a copy of the data
raw_df = data.copy()

#display first five rows of the dataset
raw_df.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,D68047AAA9A72501,classic_bike,2022-08-12 18:14:45,2022-08-12 18:21:23,Columbus Drive,JC014,Hamilton Park,JC009,40.718355,-74.038914,40.727596,-74.044247,member
1,E5B82A9F588A7650,classic_bike,2022-08-19 01:04:39,2022-08-19 01:14:10,South Waterfront Walkway - Sinatra Dr & 1 St,HB103,8 St & Washington St,HB603,40.736982,-74.027781,40.745984,-74.028199,member
2,13D19CCBCC9D7B76,classic_bike,2022-08-19 18:13:45,2022-08-19 19:32:47,South Waterfront Walkway - Sinatra Dr & 1 St,HB103,11 St & Washington St,HB502,40.736982,-74.027781,40.749985,-74.027150,member
3,AA0DBFF220DA98F6,classic_bike,2022-08-21 13:43:05,2022-08-21 13:53:13,Columbus Drive,JC014,Montgomery St,JC099,40.718355,-74.038914,40.719420,-74.050990,member
4,8F11D340492FCD32,electric_bike,2022-08-04 22:02:12,2022-08-04 22:21:31,South Waterfront Walkway - Sinatra Dr & 1 St,HB103,Pershing Field,JC024,40.736982,-74.027781,40.742677,-74.051789,member


In [3]:
raw_df.shape

(981011, 13)

### DESCRIPTIVE SUMMARY OF DATASET:
- There are a total of **981011** rows and **13** columns in the dataset.
- The type of bike are specified under **bike_type** column.
- The type of user are specified under **member_casual** column.
- The day and time when the ride started and ended are specified under **startedat** and **endedat* column.
- The **startstattionname** and **endstationname** are the names of the station where the ride started and ended.
- The**start_lat**,**start_lng**,**end_lat**,**end_lng** are the latitude and longitude points for stations.
- Each station has **start_station_id** and **end_station_id**.
- Each trip has a **ride_id**.

### DATASET INFORMATION

In [4]:
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 981011 entries, 0 to 981010
Data columns (total 13 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   ride_id             981011 non-null  object 
 1   rideable_type       981011 non-null  object 
 2   started_at          981011 non-null  object 
 3   ended_at            981011 non-null  object 
 4   start_station_name  980936 non-null  object 
 5   start_station_id    980936 non-null  object 
 6   end_station_name    977641 non-null  object 
 7   end_station_id      977641 non-null  object 
 8   start_lat           981011 non-null  float64
 9   start_lng           981011 non-null  float64
 10  end_lat             979712 non-null  float64
 11  end_lng             979712 non-null  float64
 12  member_casual       981011 non-null  object 
dtypes: float64(4), object(9)
memory usage: 97.3+ MB


#### COLUMN CONVERSION

In [5]:
#### converting starttime and endtime to datetime object
raw_df['started_at'] = pd.to_datetime(raw_df['started_at'],errors='coerce')
raw_df['ended_at'] = pd.to_datetime(raw_df['ended_at'],errors='coerce')

#### NEW COLUMN

In [6]:
# Extracting day and time for trip from start to finish
raw_df['startweekday'] = raw_df['started_at'].dt.day_name()
raw_df['endweekday'] = raw_df['ended_at'].dt.day_name()
raw_df['starttime'] = raw_df['started_at'].dt.hour
raw_df['endtime'] = raw_df['ended_at'].dt.hour
raw_df['month'] = raw_df['started_at'].dt.month_name()

# calculating the trip duration in minutes
raw_df['tripduration'] = np.round((raw_df['ended_at'] - raw_df['started_at']).dt.total_seconds()/60,2)
raw_df.head(1)


,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,startweekday,endweekday,starttime,endtime,month,tripduration
0,D68047AAA9A72501,classic_bike,2022-08-12 18:14:45,2022-08-12 18:21:23,Columbus Drive,JC014,Hamilton Park,JC009,40.718355,-74.038914,40.727596,-74.044247,member,Friday,Friday,18,18,August,6.63


#### MISSING VALUES

In [7]:
#exrtracting rows with missing value 
mask = raw_df.isnull().any(axis = 1)
missing_values = raw_df.loc[mask,:]
print(missing_values.shape[0])

#percentage of missing rows 
per_val = np.round(missing_values.shape[0] / raw_df.shape[0] * 100 ,2)
print(per_val)

# since the rows with missing values are only 0.34% of entire dataset, dropping these rows
raw_df.dropna(axis=0,inplace=True)
raw_df.shape

3370
0.34


(977641, 19)

In [8]:
 #Invalid Trip zero mins - Docking station not working properly, Starttime and endtime not recorded correctly.
new = raw_df[raw_df['tripduration'] < 1.00]
print(new.shape)

#Extracting rows with trip duration greater than zero mins
raw_df = raw_df[raw_df['tripduration'] >= 1.00]
raw_df.shape

(22750, 19)


(954891, 19)

In [9]:
#checking for duplicate records
copy_per_tripdur = raw_df.copy()
copy_per_tripdur.drop_duplicates(inplace = True)
print(copy_per_tripdur.shape)
print('There are no duplicate reecords')

(954891, 19)
There are no duplicate reecords


### ADDITIONAL DESCRIPTIVE SUMMARY OF DATASET:

- There are a total of **954891** rows and **19** columns in the dataset.
- The time when the ride started and ended are specified under **starttime** and **endtime** column.
- The **month** column states during which month the ride was taken.
- The duration of each ride is calculated under **tripduration** column.

### STATISTICAL SUMMARY

In [10]:
raw_df.describe().transpose()

,count,mean,min,25%,50%,75%,max,std
started_at,954891,2023-01-30 01:32:43.919439616,2022-08-01 00:00:07,2022-10-10 22:32:27,2023-02-03 06:50:07,2023-05-18 17:14:26.500000,2023-07-31 23:59:44,NaN
ended_at,954891,2023-01-30 01:44:34.464475392,2022-08-01 00:12:24,2022-10-10 22:51:23.500000,2023-02-03 06:56:23,2023-05-18 17:22:41,2023-08-01 08:28:23,NaN
start_lat,954891.0,40.732203,40.705897,40.721124,40.735208,40.742258,40.863943,0.01213
start_lng,954891.0,-74.040257,-74.088964,-74.045953,-74.037977,-74.031021,-73.941173,0.011989
end_lat,954891.0,40.732163,40.663062,40.721124,40.735208,40.742258,40.86448,0.012258
end_lng,954891.0,-74.039991,-74.088964,-74.045572,-74.037683,-74.03097,-73.888271,0.011991
starttime,954891.0,14.169187,0.0,10.0,15.0,18.0,23.0,5.111388
endtime,954891.0,14.265349,0.0,10.0,15.0,18.0,23.0,5.165098
tripduration,954891.0,11.842414,1.0,4.35,6.72,10.95,13621.63,46.985006


In [11]:
raw_df = raw_df[raw_df['tripduration'] <= (raw_df['tripduration'].quantile(.99))]
raw_df.shape

(945343, 19)

In [12]:
raw_df['tripduration'].describe()

count    945343.000000
mean          9.615081
std           9.641680
min           1.000000
25%           4.320000
50%           6.670000
75%          10.730000
max          77.500000
Name: tripduration, dtype: float64

In [13]:
# write to csv
raw_df.to_csv('cleaned_df.csv', index = False)